In [7]:
import tensorflow
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input,decode_predictions
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import auc

In [8]:
def vgg_model():
    model = VGG16(weights='imagenet')
    int_model = tensorflow.keras.Model(model.inputs, [model.layers[5].output, model.layers[17].output])
    return int_model


def resnet_model():
    model = ResNet50(weights='imagenet')
    int_model = tensorflow.keras.Model(model.inputs, [model.layers[14].output, model.layers[168].output])
    return int_model


def inception_model():
    model = InceptionV3(weights='imagenet')
    int_model = tensorflow.keras.Model(model.inputs, [model.layers[31].output, model.layers[284].output])
    return int_model


def features_for_img(img_path, int_model):
    img = plt.imread(img_path)[:, :, :3]
    img = tensorflow.image.resize(img, [224,224]).numpy()
    img = np.expand_dims(img, 0)
    
    low, high = int_model.predict(img)
    return low, high


def get_features_for_task(task, int_model, base_path):
    train_base_path = base_path + 'train/'
    base_path = base_path + 'test/'
    
    train_wugs_low = []
    train_wugs_high = []
    for i in range(1, 6):
        curr_path = train_base_path + str(i) + '_fin.png'
        low, high = features_for_img(curr_path, int_model)
        train_wugs_low.append(low)
        train_wugs_high.append(high)
        
    wugs_low = []
    wugs_high = []
    for i in range(1, 6):
        curr_path = base_path + 'in_' + str(i) + '_fin.png'
        low, high = features_for_img(curr_path, int_model)
        wugs_low.append(low)
        wugs_high.append(high)

    not_wugs_low_sdiff = []
    not_wugs_high_sdiff = []
    for i in range(1, 6):
        curr_path = base_path + 'out_sdiff_' + str(i) + '_fin.png'
        low, high = features_for_img(curr_path, int_model)
        not_wugs_low_sdiff.append(low)
        not_wugs_high_sdiff.append(high)
        
    not_wugs_low_ldiff = []
    not_wugs_high_ldiff = []
    for i in range(1, 6):
        curr_path = base_path + 'out_ldiff_' + str(i) + '_fin.png'
        low, high = features_for_img(curr_path, int_model)
        not_wugs_low_ldiff.append(low)
        not_wugs_high_ldiff.append(high)
        
    return train_wugs_low, train_wugs_high, wugs_low, wugs_high, not_wugs_low_sdiff, not_wugs_high_sdiff, not_wugs_low_ldiff, not_wugs_high_ldiff

def get_diff_to_mean_wug(train_wugs, wugs, not_wugs_sdiff, not_wugs_ldiff):
    mean_train_wugs = np.squeeze(np.array(train_wugs))
    mean_train_wugs = np.mean(mean_train_wugs, 0)
       
    wugs_diff = []
    for e in wugs:
        wugs_diff.append(np.sum(np.abs(e - mean_train_wugs)))
      
    not_wugs_sdiff_diff = []
    for e in not_wugs_sdiff:
        not_wugs_sdiff_diff.append(np.sum(np.abs(e - mean_train_wugs)))
        
    not_wugs_ldiff_diff = []
    for e in not_wugs_ldiff:
        not_wugs_ldiff_diff.append(np.sum(np.abs(e - mean_train_wugs)))
    
    return wugs_diff, not_wugs_sdiff_diff, not_wugs_ldiff_diff

def tpr_fpr_for_threshold(wug, not_wug, threshold):
    pos = 0
    for d in wug:
        if d <= threshold:
            pos += 1
    tpr = pos / len(wug)
    
    neg_wrong = 0
    for d in not_wug:
        if d <= threshold:
            neg_wrong += 1
    fpr = neg_wrong / len(not_wug)
    
    return tpr, fpr


def get_auc(wugs_diff, not_wugs_diff):
    all_diffs = sorted(wugs_diff + not_wugs_diff)
    tprs = []
    fprs = []
    ts = all_diffs
    for t in ts:
        tpr, fpr = tpr_fpr_for_threshold(wugs_diff, not_wugs_diff, t)
        tprs.append(tpr)
        fprs.append(fpr)
    curr_auc = auc(fprs, tprs)
    return curr_auc, fprs, tprs
    

def eval_for_task(task, int_model, base_path):
    train_wugs_low, train_wugs_high, wugs_low, wugs_high, not_wugs_low_sdiff, not_wugs_high_sdiff, not_wugs_low_ldiff, not_wugs_high_ldiff = get_features_for_task(task, int_model, base_path)
    
    # Low level features
    wugs_low_diff, not_wugs_sdiff_low_diff, not_wugs_ldiff_low_diff = get_diff_to_mean_wug(train_wugs_low, wugs_low, not_wugs_low_sdiff, not_wugs_low_ldiff)
    
    auc_low, _, _, = get_auc(wugs_low_diff, not_wugs_sdiff_low_diff + not_wugs_ldiff_low_diff)
    auc_low_sdiff, _, _, = get_auc(wugs_low_diff, not_wugs_sdiff_low_diff)
    auc_low_ldiff, _, _, = get_auc(wugs_low_diff, not_wugs_ldiff_low_diff)
    
    # High level features
    wugs_high_diff, not_wugs_sdiff_high_diff, not_wugs_ldiff_high_diff = get_diff_to_mean_wug(train_wugs_high, wugs_high, not_wugs_high_sdiff, not_wugs_high_ldiff)
    
    auc_high, _, _, = get_auc(wugs_high_diff, not_wugs_sdiff_high_diff + not_wugs_ldiff_high_diff)
    auc_high_sdiff, _, _, = get_auc(wugs_high_diff, not_wugs_sdiff_high_diff)
    auc_high_ldiff, _, _, = get_auc(wugs_high_diff, not_wugs_ldiff_high_diff)
    
    print('AUC low-level features: ' + str(round(auc_low, 4)) + '\n')
    print('AUC low-level features: (sdiff): ' + str(round(auc_low_sdiff, 4)) + '\n')
    print('AUC low-level features (ldiff): ' + str(round(auc_low_ldiff, 4)) + '\n')
    print('\n')
    print('AUC high-level features: ' + str(round(auc_high, 4)) + '\n')
    print('AUC high-level features (sdiff): ' + str(round(auc_high_sdiff, 4)) + '\n')
    print('AUC high-level features (ldiff): ' + str(round(auc_high_ldiff, 4)) + '\n')
    print('\n')



In [9]:
int_model = vgg_model()

In [10]:
eval_for_task('concept_cct', int_model, '/viscam/u/joycj/geoclidean/dataset/z_geo_dataset/constraints/concept_cct/')





AUC low-level features: 0.48

AUC low-level features: (sdiff): 0.44

AUC low-level features (ldiff): 0.52



AUC high-level features: 0.78

AUC high-level features (sdiff): 0.68

AUC high-level features (ldiff): 0.88



